# Intro

PostgreSQL, also known as Postgres, is an extremely powerful database engine. At a high level, PostgreSQL consists of two pieces, a server and clients. The server is a program that manages databases and handles queries.



By default, PostgreSQL uses port 5432 to communicate with the outside world. If you start a PostgreSQL server, it will listen for incoming connections on port 5432. Clients will be able to connect to the server using this port. If you start a client, you'll have to specify which server to connect to, along with the port to connect to.

## Psycopg2

There are many clients for PostgreSQL, including graphical clients. The most common Python client for PostgreSQL is called psycopg2. Connecting to a PostgreSQL database using psycopg2 is similar to connecting to a SQLite database using the sqlite3 libary. psycopg2 also uses Connection and Cursor objects.



In [3]:
import psycopg2
conn = psycopg2.connect("dbname=postgres user=postgres")
cur = conn.cursor()

OperationalError: could not connect to server: No such file or directory
	Is the server running locally and accepting
	connections on Unix domain socket "/var/run/postgresql/.s.PGSQL.5432"?


In [4]:
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()
cur.execute("CREATE TABLE notes (id integer PRIMARY KEY,body text,title text);")

conn.close()


OperationalError: could not connect to server: No such file or directory
	Is the server running locally and accepting
	connections on Unix domain socket "/var/run/postgresql/.s.PGSQL.5432"?


# SQL Transactions

If you checked the database dq now, you would notice that there actually isn't a notes table inside it. This isn't a bug -- it's because of a concept called SQL transactions. With SQLite, every query we made that modified the data was immediately executed, and immediately changed the database.

Transactions prevent this type of behavior by **ensuring that all the queries in a transaction block are executed at the same time**. If **any of the transactions fail, the whole group fails**, and no changes are made to the database at all.

Whenever we open a Connection in **psycopg2**, a new transaction will automatically be created. All queries run up until the commit method is called will be placed into the same transaction block. When **commit is called**, the PostgreSQL engine will run all the queries at once.

If we don't want to apply the changes in the transaction block, we can call the rollback method to remove the transaction. **Not calling either commit or rollback** will cause the transaction to **stay in a pending state**, and will result in the changes not being applied to the database.

```SQL
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()
cur.execute("CREATE TABLE notes (id integer PRIMARY KEY, body text, title text);")
conn.commit()
conn.close()
```

## Autocommit

Some changes also have such widespread effects that they can't be wrapped inside of a transaction. **One example of this is creating a database**. When creating a database, we'll need to **activate autocommit mode** first.

```SQL
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
conn.autocommit = True   // sets autommit to true
cur = conn.cursor()
cur.execute("CREATE TABLE notes (id integer PRIMARY KEY, body text, title text);")
conn.commit()
conn.close()
```

## Fetchone () vs. fetchall

We can issue SELECT queries against a database using the execute method, along with the fetchall and fetchone methods to retrieve results:

```SQL
#connection and cursor already established

cur.execute("INSERT INTO notes VALUES (2, 'Do more missions on Dataquest', 'Dataquest reminder');")

cur.execute("SELECT * FROM notes")
rows = cur.fetchall()
print(rows)

conn.commit()
conn.close
```


## Creating a PostgreSQL database

One of the most powerful aspects of PostgreSQL is that it enables you to create multiple databases. Different databases are generally used to hold information about different applications, i.e.
* an application to save and read notes
* income information
* ..

By storing data for a single application in a single database, we encapsulate that application, and make it easier to manage and alter the data for it.

We can create a DB with the followin SQL command:

```SQL
CREATE DATABASE dbName;
```

We can specify the user who will own the database when we create it as well, using the OWNER statement:


```SQL
CREATE DATABASE dbName OWNER postgres;
```


## Deleting a DB

```SQL
DROP DATABASE DbName;
```

e.g. 
```SQL
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("DROP DATABASE income;")
conn.close()
```